<a href="https://colab.research.google.com/github/walkershashi/FourSquare-API/blob/master/Searching_for_Specific_Venue_Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Installed')

Installed


In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [ ]:
CLIENT_ID = 'OECZ01BUCTDUDGR0YUA4VKHV1T4COOU5535O5OKPCJFEAU3F' # your Foursquare ID
CLIENT_SECRET = 'FETLWMYN0AT51PPEKTIYBFRENECKX1OBTUMNN5U2BHI3TH0K' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [ ]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7151482 -74.0156573


In [ ]:
search_query = 'gym'
radius = 500
print(search_query + ' .... OK!')

gym .... OK!


In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
#url

In [ ]:
results = requests.get(url).json()
#results

In [ ]:
venues = results['response']['venues']
venues_df = json_normalize(venues)
venues_df.to_csv(index = False)
venues_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c8f73d24e95236ac70084ca,Gym @ St. John's,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1595668893,False,101 Murray St,40.715446,-74.012627,"[{'label': 'display', 'lat': 40.71544647216797...",257,10007,US,New York,NY,United States,"[101 Murray St, New York, NY 10007, United Sta...",NaN,NaN
1,5a595db6fe374079de98f6b5,Gym at Battery Park City Community Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1595668893,False,NaN,40.717914,-74.013884,"[{'label': 'display', 'lat': 40.717914, 'lng':...",342,10282,US,New York,NY,United States,"[New York, NY 10282, United States]",NaN,NaN
2,4c8d0269f87e224bf4c13905,89 Murray Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595668893,False,89 Murray St,40.715630,-74.012115,"[{'label': 'display', 'lat': 40.71562957763672...",303,10007,US,New York,NY,United States,"[89 Murray St, New York, NY 10007, United States]",NaN,NaN
3,4cc1c0a667a3b1f7c29fca0e,Stuyvesant High School (3rd floor gym),"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",v-1595668893,False,345 Chambers St,40.717884,-74.013908,"[{'label': 'display', 'lat': 40.7178840637207,...",338,10282,US,New York,NY,United States,"[345 Chambers St, New York, NY 10282, United S...",NaN,NaN
4,4c0d68c8336220a1c2adca77,Millenium Hilton Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595668893,False,55 Church St,40.711673,-74.011149,"[{'label': 'display', 'lat': 40.71167268972019...",542,10007,US,New York,NY,United States,"[55 Church St, New York, NY 10007, United States]",NaN,NaN


In [ ]:
venues_id = venues_df[['id']]
venues_df = venues_df.drop(['id', 'referralId', 'hasPerk', 'location.labeledLatLngs', 'location.cc', 'location.crossStreet', 'venuePage.id'],axis = 1)
venues_df = pd.concat([venues_df, venues_id], axis = 1)

venues_df.head()

,name,categories,location.address,location.lat,location.lng,location.distance,location.postalCode,location.city,location.state,location.country,location.formattedAddress,id
0,Gym @ St. John's,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",101 Murray St,40.715446,-74.012627,257,10007,New York,NY,United States,"[101 Murray St, New York, NY 10007, United Sta...",4c8f73d24e95236ac70084ca
1,Gym at Battery Park City Community Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",NaN,40.717914,-74.013884,342,10282,New York,NY,United States,"[New York, NY 10282, United States]",5a595db6fe374079de98f6b5
2,89 Murray Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",89 Murray St,40.715630,-74.012115,303,10007,New York,NY,United States,"[89 Murray St, New York, NY 10007, United States]",4c8d0269f87e224bf4c13905
3,Stuyvesant High School (3rd floor gym),"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",345 Chambers St,40.717884,-74.013908,338,10282,New York,NY,United States,"[345 Chambers St, New York, NY 10282, United S...",4cc1c0a667a3b1f7c29fca0e
4,Millenium Hilton Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",55 Church St,40.711673,-74.011149,542,10007,New York,NY,United States,"[55 Church St, New York, NY 10007, United States]",4c0d68c8336220a1c2adca77


In [ ]:
filtered_columns = ['name', 'categories'] + [col for col in venues_df.columns if col.startswith('location.')] + ['id']
columns = [column.split('.')[-1] for column in filtered_columns]
        
venues_df.columns = columns

for i in range(venues_df.shape[0]):
    venues_df['categories'][i] = venues_df['categories'][i][0]['name']

venues_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,name,categories,address,lat,lng,distance,postalCode,city,state,country,formattedAddress,id
0,Gym @ St. John's,Gym / Fitness Center,101 Murray St,40.715446,-74.012627,257,10007,New York,NY,United States,"[101 Murray St, New York, NY 10007, United Sta...",4c8f73d24e95236ac70084ca
1,Gym at Battery Park City Community Center,Gym / Fitness Center,NaN,40.717914,-74.013884,342,10282,New York,NY,United States,"[New York, NY 10282, United States]",5a595db6fe374079de98f6b5
2,89 Murray Gym,Gym,89 Murray St,40.715630,-74.012115,303,10007,New York,NY,United States,"[89 Murray St, New York, NY 10007, United States]",4c8d0269f87e224bf4c13905
3,Stuyvesant High School (3rd floor gym),College Library,345 Chambers St,40.717884,-74.013908,338,10282,New York,NY,United States,"[345 Chambers St, New York, NY 10282, United S...",4cc1c0a667a3b1f7c29fca0e
4,Millenium Hilton Gym,Gym,55 Church St,40.711673,-74.011149,542,10007,New York,NY,United States,"[55 Church St, New York, NY 10007, United States]",4c0d68c8336220a1c2adca77


In [ ]:
# Add a marker to the current location

venues_map = folium.Map(location = [latitude, longitude], zoom_start = 15)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color = 'red',
    popup = 'Your Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


venues_map

In [ ]:
# Display marker for the differnet venues

for lat, lng, label in zip(venues_df.lat, venues_df.lng, venues_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'yellow',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6
    ).add_to(venues_map)
    
# Add pop-up text to each marker
LATITUDE, LONGITUDE = list(venues_df.lat), list(venues_df.lng)
NAME = list(venues_df.name)

for lat, lng, label in zip(LATITUDE, LONGITUDE, NAME):
    folium.Marker(
        [lat, lng],
        popup = label
    ).add_to(venues_map)
    
venues_map